In [1]:
import pandas as pd
from sqlite_utils import Database

In [2]:
db = Database("mybinder_archive.db")

In [3]:
# get table names
db.table_names()

['mybinderlaunch', 'repo', 'notebook']

In [4]:
launch_table = 'mybinderlaunch'

In [5]:
# db[launch_table].columns
db[launch_table].columns_dict

{'timestamp': str,
 'version': int,
 'origin': str,
 'provider': str,
 'spec': str,
 'org': str,
 'ref': str,
 'resolved_ref': str,
 'r2d_version': str,
 'repo_url': str,
 'repo_id': str}

In [6]:
# NOTE: this notebook wont have same output (nbs_same_output)
query = f'SELECT * FROM {launch_table} ORDER BY RANDOM() LIMIT 100'
pd.read_sql_query(query, db.conn)

,timestamp,version,origin,provider,spec,org,ref,resolved_ref,r2d_version,repo_url,repo_id
0,2019-10-19T11:59:00,3,ovh.mybinder.org,GitHub,jupyterlab/jupyterlab-demo/try.jupyter.org,jupyterlab,try.jupyter.org,,jupyter/repo2docker:0.10.0-100.g0dba611,https://github.com/jupyterlab/jupyterlab-demo,4
1,2019-11-29T19:40:00,3,notebooks.gesis.org,GitHub,binder-examples/requirements/master,binder-examples,master,,jupyter/repo2docker:0.10.0-117.g3337eaa,https://github.com/binder-examples/requirements,165
2,2019-09-16T13:38:00,3,gke.mybinder.org,GitHub,ipython/ipython-in-depth/master,ipython,master,,jupyter/repo2docker:0.10.0-59.g649a2c6,https://github.com/ipython/ipython-in-depth,1
3,2019-09-16T11:24:00,3,gke.mybinder.org,GitHub,freechipsproject/chisel-bootcamp/master,freechipsproject,master,,jupyter/repo2docker:0.10.0-59.g649a2c6,https://github.com/freechipsproject/chisel-boo...,6702
4,2019-11-06T04:45:00,3,notebooks.gesis.org,GitHub,jupyterlab/jupyterlab-demo/try.jupyter.org,jupyterlab,try.jupyter.org,,jupyter/repo2docker:0.10.0-100.g0dba611,https://github.com/jupyterlab/jupyterlab-demo,4
...,...,...,...,...,...,...,...,...,...,...,...
95,2020-06-01T19:27:00,3,gke.mybinder.org,GitHub,binder-examples/r/master,binder-examples,master,,jupyter/repo2docker:0.11.0-72.gbd72e16,https://github.com/binder-examples/r,17
96,2019-06-09T11:41:00,2,mybinder.org,GitHub,thpclass/simple-substrate/master,thpclass,master,,jupyter/repo2docker:f19e159d,https://github.com/thpclass/simple-substrate,16636
97,2020-05-22T11:34:00,3,gke.mybinder.org,Gist,ELC/8fdc0f490b3058872a7014f01416dfb6/master,ELC,master,,jupyter/repo2docker:0.11.0-66.gd03c283,https://gist.github.com/elc/8fdc0f490b3058872a...,12704
98,2019-10-30T20:48:00,3,gke.mybinder.org,GitHub,ipython/ipython-in-depth/master,ipython,master,,jupyter/repo2docker:0.10.0-100.g0dba611,https://github.com/ipython/ipython-in-depth,1


### Total number of launches

In [7]:
db[launch_table].count

10081604

First timestamp

In [8]:
launches = db[launch_table].rows_where(order_by="timestamp")
first_launch_timestamp = next(launches)["timestamp"]
del launches

first_launch_timestamp

'2018-11-03T00:00:00'

Last timestamp

In [9]:
launches = db[launch_table].rows_where(order_by="timestamp desc")
last_launch_timestamp = next(launches)["timestamp"]
del launches

last_launch_timestamp

'2020-07-26T23:59:00'

### Launches per origin

In [10]:
query = f'SELECT origin, count(origin) FROM {launch_table} GROUP BY "origin";'
launches_per_origin = pd.read_sql_query(query, db.conn)
launches_per_origin

,origin,count(origin)
0,binder.mybinder.ovh,585
1,binder.mybinder.turing.ac.uk,137
2,gesis.mybinder.org,830943
3,gke.mybinder.org,5013540
4,gke.mybinder.org:443,1
5,mybinder.org,2741924
6,notebooks.gesis.org,670816
7,ovh.mybinder.org,564270
8,turing.mybinder.org,259388


In [11]:
launches_per_origin_dict = {"gke": 0, "gesis": 0, "turing": 0, "ovh": 0}
for index, row in launches_per_origin.iterrows():
    if row["origin"] == "mybinder.org":
        launches_per_origin_dict["gke"] += row["count(origin)"]
    for origin in launches_per_origin_dict.keys():
        if origin in row["origin"]:
            # print(origin, row["origin"])
            launches_per_origin_dict[origin] += row["count(origin)"]

assert sum(launches_per_origin_dict.values()) == db[launch_table].count

pd.DataFrame.from_dict(
    launches_per_origin_dict, 
    orient="index", 
    columns=["launches"]
).sort_values(by=["launches"], ascending=False)

,launches
gke,7755465
gesis,1501759
ovh,564855
turing,259525


### Launches per provider

In [12]:
query = f'SELECT provider, count(provider) FROM {launch_table} GROUP BY "provider";'
launches_per_provider = pd.read_sql_query(query, db.conn).sort_values(by=["count(provider)"], ascending=False)

launches_per_provider

,provider,count(provider)
4,GitHub,9826135
2,Gist,141237
3,Git,70747
5,GitLab,41629
7,Zenodo,1025
6,Hydroshare,393
1,Figshare,309
0,Dataverse,129


In [13]:
# percentage of first 2 providers
(sum(launches_per_provider.head(2)["count(provider)"])*100)/sum(launches_per_provider["count(provider)"])

98.86692633434124

### Repos per provider

In [14]:
query = f'SELECT provider, count(DISTINCT repo_url) FROM {launch_table} GROUP BY "provider";'
repos_per_provider = pd.read_sql_query(query, db.conn).\
                        sort_values(by=["count(DISTINCT repo_url)"], ascending=False)

repos_per_provider

,provider,count(DISTINCT repo_url)
4,GitHub,51062
3,Git,10598
2,Gist,1577
5,GitLab,936
7,Zenodo,92
6,Hydroshare,39
1,Figshare,30
0,Dataverse,14


In [15]:
# # this returns 197 repos for Git provider because they have github url but Git as provider, 
# # which is okay and should work 
# query = f'SELECT provider, count(DISTINCT repo_id) FROM {launch_table} GROUP BY "provider";'
# repos_per_provider = pd.read_sql_query(query, db.conn).\
#                         sort_values(by=["count(DISTINCT repo_id)"], ascending=False)

# repos_per_provider

### Organisations/Users in repos

In [16]:
# number of different orgs
query = "select count(distinct org) from mybinderlaunch;"
db.conn.execute(query).fetchone()[0]

34986